<img src="image/tableau_meta.png" alt="Tableau Meta" width="300" height="300" style="float:left; margin-right: 40px; margin-bottom: 20px;" />

# Tableau Metadata AI Notebook
* #### This notebook provides methods in Python to leverage Tableau's APIs to:
1. Extract details for Tableau server workbooks and data sources/fields.
2. Parse Tableau workbook metadata/XML enabling AI translations to describe workbooks.
3. Utilize AI meta translation to build documentation resources.


<div style="clear: both;"></div>


# Table of Contents
- [Prerequisites](#prerequisite)
   - [OpenAI API Key](#openai-api-key) 
   - [Tableau Server (optional)](#tableau-server-optional) 
1. [Step 1 Set Up](#step-1-set-up)
2. [Step 2 Testing API Connections](#step-2-testing-api-connections)
   - [OpenAI API](#openai-api)
   - [Tableau Server API (optional)](#tableau-server-api-optional) 
3. [Step 3 Walk Through](#step-3-walk-through)
   - [3.01 Get All Datasources](#301-get-all-datasources) 
   - [3.02 Get All Workbooks](#302-get-all-workbooks)  
   - [3.03 Download Workbook and Parse XML](#303-download-workbook-and-parse-xml)  
   - [3.04 Get Worksheet Meta](#304-get-worksheet-meta)
   - [3.05 Get Dashboard Meta](#305-get-dashboard-meta) 
   - [3.06 Get Worksheets in Dashboard](#306-get-worksheets-in-dashboard)
   - [3.07 Get XML](#307-get-xml)
   - [3.08 Helper Functions](#308-helper-functions)
   - [3.09 Preprocessing](#309-preprocessing)
   - [3.10 AI Processing](#310-ai-preprocessing)



For details on Tableau API methods, see the [Tableau REST API Reference](https://help.tableau.com/current/api/rest_api/en-us/REST/rest_api.htm).

For details on OpenAI API methods, see the [OpenAI API Reference](https://platform.openai.com/docs/api-reference/introduction).


<h1 id="prerequisites">Prerequisites</h1> <a id="prerequisites"></a>

<h2 id="openai-api-key">OpenAI API Key</h2> <a id="openai-api-key"></a> 
To use the notebook as currently configured, you will need an API key from OpenAI. However, with minor adjustments to the codebase, you can theoretically use any generative AI tool. The codebase currently limits the token count for AI processing to 10,000 tokens. For more efficient results, it is recommended to use a model that can handle up to 100,000 tokens, allowing for processing larger inputs or generating longer outputs.


<h2 id="tableau-server-optional">Tableau Server (optional)</h2> <a id="tableau-server-optional"></a>
This notebook includes operations to support data extractions from the server, but Tableau workbooks can also be processed locally by adjusting the file paths within the codebase. If you are using a Tableau server, please note that your access to products and data sources may be limited based on your site permissions.

<h1 id="step-1-set-up">Step 1 Set Up</h1> <a id="step-1-set-up"></a>

#### Steps to Setup
1. Create a virtual environment for the project
2. Start the notebook in that virtual environment
3. Uncomment the line { !pip install -r requirements.txt } and run the cell to install the required packages then comment that line out when completed
4. Import the packages into the notebook

Note: This assumes you’re running the notebook from a local machine and not a shared resource like CoLab. 


In [ ]:
#!pip install -r requirements.txt

In [367]:
import tableauserverclient as TSC
from tableauserverclient import TableauAuth, Server, RequestOptions
from tableaudocumentapi import Workbook
from tableaudocumentapi.xfile import xml_open
import lxml.etree as etree
import openai  # for using GPT and getting embeddings
import tiktoken # counting tokens, used by openai 
import requests
import os
import re
import pandas as pd



<h1 id="step-2-testing-api-connections">Step 2 Testing API Connections</h1> <a id="step-2-testing-api-connections"></a>

#### Steps to Test Connections
1. OpenAI and Tableau connections will be tested separately
2. Update the respective variables for each connection
3. Run the respective cells

<h2 id="openai-api">OpenAI API</h2> <a id="openai-api"></a>

#### Testing your OpenAI API connection
- Update the API key and AI model (if needed) then run the respective cells
- The code should print the error response allowing you to troubleshoot. Expected issues would be either your access to particular models or issues with your API key permissions.

> **Action**: Update the following variables

In [379]:
# update the following variables 
model = "gpt-4-1106-preview"
api_key = 'YOUR KEY HERE'


> **Action**: Run the following test cell to determine if there are issues with your OpenAI API key

In [ ]:

# Set up your OpenAI client
openai.api_key = api_key

# A test prompt to send to the OpenAI API
test_prompt = "Does this work?"

# API endpoint URL
api_url = 'https://api.openai.com/v1/chat/completions'

# Request headers
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}


data = {
    'model': model,
    'messages': [{'role': 'system', 'content': "You are a helpful assistant."},
                    {'role': 'user', 'content': test_prompt}]
}

# Make the API call
response = requests.post(api_url, json=data, headers=headers)

# Process the API response
if response.status_code == 200:
    print('success')
    result = response.json()
    chatgpt_response = result['choices'][0]['message']['content']

else:
    print('error')
    chatgpt_response = f"'Error:', {response.status_code}, {response.text}"
print(chatgpt_response)

<h2 id="tableau-server-api-optional">Tableau Server API (optional)</h2> <a id="tableau-server-api-optional"></a>

#### Testing your Tableau Server API connection
- Update the login variables then run the respective cells
- The code should print the error response allowing you to troubleshoot. Expected issues would be either your credentials or if your server requires you to be on a network.
- This is listed as optional as you can use a Tableau workbook on your local machine by replacing "file_path" with the path to your file and and running that line of code down to the end of the cell by itself.

> **Action**: Update the following variables

In [ ]:
# update the following variables 
user_name = 'USER NAME'
pw = 'PASSWORD'
site_name = 'SITE-NAME'
server = 'https://YOUR-SERVER.name'


> **Action**: Run the following test cell to determine if there are issues with your Tableau API access

In [ ]:

tableau_auth = TSC.TableauAuth(user_name, pw, site_name)
server = TSC.Server(server, use_server_version=True)
# server.version = '3.7'

try:
    with server.auth.sign_in(tableau_auth):
        # Retrieve server information
        server_info = server.server_info.get()
        print("Successfully connected to the server!")
        print("Server version:", server_info.product_version)

        # Optionally, list available projects to verify access permissions
        all_projects, pagination_item = server.projects.get()
        print("\nProjects accessible to you:")
        for project in all_projects:
            print("-", project.name)
except TSC.ServerResponseError as e:
    print("Error connecting to the server:", e)

<h1 id="step-3-walk-through">Step 3 Walk Through</h1> <a id="step-3-walk-through"></a>
  

<h2 id="301-get-all-datasources">3.01 Get All Datasources</h2> <a id="301-get-all-datasources"></a>

##### Get all datasources from your Tableau site to include source IDs

In [ ]:

data_sources_details = []

with server.auth.sign_in(tableau_auth):
    all_datasources, pagination_item = server.datasources.get()
    print("\nThere are {} datasources on site: ".format(pagination_item.total_available))

    for datasource in all_datasources:
        # Populate connections for the data source
        server.datasources.populate_connections(datasource)

        for connection in datasource.connections:
            # Append details to the list
            data_sources_details.append({
                "Data Source Name": datasource.name,
                "Data Source ID": datasource.id,
                "Connection Type": getattr(connection, 'connection_type', 'N/A'),
                "Connection ID": connection.id
            })

# Convert the list of dictionaries into a DataFrame
tableau_data_sources = pd.DataFrame(data_sources_details)

<h2 id="302-get-all-workbooks">3.02 Get All Workbooks</h2> <a id="302-get-all-workbooks"></a>

##### Get all workbooks from your Tableau site with helpful meta...this one could take some time to run

In [ ]:

# Sign in to the Tableau Server
with server.auth.sign_in(tableau_auth):
    workbooks_info = []  # To store workbook details

    # Use TSC.Pager to handle pagination automatically
    pager = TSC.Pager(server.workbooks)  # Automatically handles pagination

    for workbook in pager:
        try:
            # Populate additional details for each workbook
            server.workbooks.populate_connections(workbook)
            server.workbooks.populate_views(workbook)

            # Extract the required details
            workbook_details = {
                'workbook_name': workbook.name,
                'id': workbook.id,
                'project_name': workbook.project_name,
                'project_id': workbook.project_id,
                'connections': [connection.id for connection in workbook.connections],
                'wookbook_tags': list(workbook.tags),  # Convert to list of strings
                'workbook_views': [view.name for view in workbook.views],
                'webpage_url': workbook.webpage_url
            }
            workbooks_info.append(workbook_details)

        except Exception as e:
            print(f"Issue with workbook {workbook.id}: {str(e)}")

    # Create a DataFrame from the workbook info
    workbooks_df = pd.DataFrame(workbooks_info)


    print(f"Total workbooks processed: {len(workbooks_df)}")


<h2 id="303-download-workbook-and-parse-xml">3.03 Download Workbook and Parse XML</h2> <a id="303-download-workbook-and-parse-xml"></a>

##### Get your target workbook from the server and process the XML. Note: os.remove(file_path) has been commented out so update as needed.

> **Action**: Update the variable { target_workbook } in the next cell with the title of workbook you would like to use for testing this process.

In [ ]:
target_workbook = 'YOUR WORKBOOK NAME HERE'

find = workbooks_df[workbooks_df.workbook_name == target_workbook] 
find_views = find.iloc[0]['workbook_views']

# get your workbook id and project id to extract the required workbook
project_id = find.iloc[0]['project_id'] 
workbook_id = find.iloc[0]['id'] 

> **Action**: Run the next cell to download your workbook from the server and parse the XML meta

In [ ]:
# download your workbook from the server and parse the XML meta 

def get_workbook_details(workbook_id, project_id, server, tableau_auth):   
    with server.auth.sign_in(tableau_auth):
        for workbook in TSC.Pager(server.workbooks):
            # Match the workbook based on the workbook id and project id
            if workbook.id == workbook_id and workbook.project_id == project_id:
                return {
                    "Workbook Id": workbook.id,
                    "Project Id": workbook.project_id,
                    "Workbook Webpage_Url": workbook.webpage_url,
                    "Workbook Name": workbook.name
                }
        
        print("No workbook found with the specified workbook ID and project ID.")
        return {}




#Locate workbook
source_workbook = get_workbook_details(workbook_id, project_id, server, tableau_auth)
 
 
with server.auth.sign_in(tableau_auth):    
    file_path = server.workbooks.download(source_workbook["Workbook Id"], filepath=None, include_extract=True, no_extract=None)
    print("\nDownloaded the file to {0}.".format(file_path))

    #*****Get list of active views and Dashboards from Worbook*****
    views_to_include = []
 
    workbook_item = server.workbooks.get_by_id(source_workbook["Workbook Id"])
    server.workbooks.populate_views(workbook_item)
    for item in workbook_item.views:
        views_to_include.append(item.name)



#*****Get list of all views and Dashboards from Worbook***** 
 
#Load Workbook from Document API
sourceWB = Workbook(file_path)
 
#Get a list of all worksheets and dashboards
all_views = sourceWB.dashboards + sourceWB.worksheets
 
#Views not made visiable on the server
views_to_hide = []
 
for view in all_views:
    if view not in views_to_include:
        views_to_hide.append(view)
 
 
#Remove downloaded file
#os.remove(file_path)
 
print("\n\n")


print(views_to_hide)

<h2 id="304-get-worksheet-meta">3.04 Get Worksheet Meta</h2> <a id="304-get-worksheet-meta"></a>
 
##### Get the worksheets from the Tableau workbook in a dataframe with meta via fields

In [353]:


# Open the XML file
xml_tree = xml_open(file_path)

# Get the root element (the top-level element of the XML document)
root = xml_tree.getroot()


worksheets = root.findall(".//worksheet")

df_worksheet_hold = []

for worksheet in worksheets:
    # Print the worksheet name
    worksheet_name = worksheet.get('name')
    #print(f"Worksheet: {worksheet_name}")
    
    # Find all 'datasource' elements within the 'datasources' tag under each worksheet
    datasources = worksheet.findall(".//datasources/datasource")
    
    for datasource in datasources:
        # Get the 'caption' or 'name' attribute of the datasource
        caption = datasource.get('caption', 'No Caption')  # Use a default if 'caption' doesn't exist
        name = datasource.get('name', 'No Name')  # Use a default if 'name' doesn't exist
        #print(f"  Datasource: Caption={caption}, Name={name}")
        
        # Find columns in the datasource-dependencies for the current datasource
        datasource_dependencies = worksheet.findall(f".//datasource-dependencies[@datasource='{name}']/column")


        # Handle columns
        for column in datasource_dependencies:
            # Get column attributes
            column_name =  re.sub('\]|\[','', column.get('name', 'No Name'))
            column_caption = column.get('caption', 'No Caption')
            column_datatype = column.get('datatype', 'No Datatype')
            column_aggregation = column.get('aggregation', 'No Aggregation')
            column_role = column.get('role', 'No Role')
            column_formula = column.find(".//calculation").get('formula', 'No Formula') if column.find(".//calculation") is not None else 'No Formula'
            
            # Print the column details
            #print(f"    Column Name: {column_name}, Caption: {column_caption}, Datatype: {column_datatype}, Aggregation: {column_aggregation}, Role: {column_role}")
            #print(f"    Formula: {column_formula}")
            data = {'type':'worksheet',
                    'title':worksheet_name,
                    'datasource':caption,
                    'column_name':column_caption,
                    'formula':column_formula}
            df_temp1 = pd.DataFrame([data])
            df_worksheet_hold.append(df_temp1)
        
        # Find column instances for the current datasource
        column_instances = worksheet.findall(f".//datasource-dependencies[@datasource='{name}']/column-instance")


        # Handle column instances
        for column_instance in column_instances:
            # Get column-instance attributes
            column_instance_name = column_instance.get('name', 'No Name')
            column_instance_base_column = re.sub('\]|\[','', column_instance.get('column', 'No Base Column'))
            column_instance_derivation = column_instance.get('derivation', 'No Derivation')
            column_instance_type = column_instance.get('type', 'No Type')

            # Print the column-instance details
            #print(f"    Column Instance: {column_instance_name}, Base Column: {column_instance_base_column}, Derivation: {column_instance_derivation}, Type: {column_instance_type}")

            data = {'type':'worksheet',
                    'title':worksheet_name,
                    'datasource':caption,
                    'column_name':column_instance_base_column,}
            df_temp2 = pd.DataFrame([data])
            df_worksheet_hold.append(df_temp2)


df_worksheet_concat = pd.concat(df_worksheet_hold).reset_index(drop=True)


<h2 id="305-get-dashboard-meta">3.05 Get Dashboard Meta</h2> <a id="305-get-dashboard-meta"></a>


##### Get the dashboards from the Tableau workbook in a dataframe with meta on fields used

In [ ]:
# Find all dashboard elements
dashboards = root.findall(".//dashboard")

df_dashoard_hold = []

for dashboard in dashboards:
    # Print the dashboard name
    dashboard_name = dashboard.get('name')
    #print(f"Dashboard: {dashboard_name}")
    
    # Find all 'datasource' elements within the 'datasources' tag under each dashboard
    datasources = dashboard.findall(".//datasources/datasource")
    
    for datasource in datasources:
        # Get the 'caption' or 'name' attribute of the datasource
        caption = datasource.get('caption', 'No Caption')  # Use a default if 'caption' doesn't exist
        name = datasource.get('name', 'No Name')  # Use a default if 'name' doesn't exist
        #print(f"  Datasource: Caption={caption}, Name={name}")
        
        # Find columns in the datasource-dependencies for the current datasource
        datasource_dependencies = dashboard.findall(f".//datasource-dependencies[@datasource='{name}']/column")

        # Handle columns
        for column in datasource_dependencies:
            # Get column attributes
            column_name = re.sub(r'\]|\[', '', column.get('name', 'No Name'))
            column_caption = column.get('caption', 'No Caption')
            column_datatype = column.get('datatype', 'No Datatype')
            column_aggregation = column.get('aggregation', 'No Aggregation')
            column_role = column.get('role', 'No Role')
            column_formula = column.find(".//calculation").get('formula', 'No Formula') if column.find(".//calculation") is not None else 'No Formula'
            
            # Print the column details
            #print(f"    Column Name: {column_name}, Caption: {column_caption}, Datatype: {column_datatype}, Aggregation: {column_aggregation}, Role: {column_role}")
            #print(f"    Formula: {column_formula}")
            
            # Prepare data for dataframe
            data = {
                'type': 'dashboard',
                'title': dashboard_name,
                'datasource': caption,
                'column_name': column_name,
                'formula': column_formula
            }
            df_temp1 = pd.DataFrame([data])
            df_dashoard_hold.append(df_temp1)
        
        # Find column instances for the current datasource
        column_instances = dashboard.findall(f".//datasource-dependencies[@datasource='{name}']/column-instance")

        # Handle column instances
        for column_instance in column_instances:
            # Get column-instance attributes
            column_instance_name = column_instance.get('name', 'No Name')
            column_instance_base_column = re.sub(r'\]|\[', '', column_instance.get('column', 'No Base Column'))
            column_instance_derivation = column_instance.get('derivation', 'No Derivation')
            column_instance_type = column_instance.get('type', 'No Type')

            # Print the column-instance details
            #print(f"    Column Instance: {column_instance_name}, Base Column: {column_instance_base_column}, Derivation: {column_instance_derivation}, Type: {column_instance_type}")
            
            # Prepare data for dataframe
            data = {
                'type': 'dashboard',
                'title': dashboard_name,
                'datasource': caption,
                'column_name': column_instance_name,
                'column_instance_base_column':column_instance_base_column
            }
            df_temp2 = pd.DataFrame([data])
            df_dashoard_hold.append(df_temp2)

# Concatenate all dataframes into one
df_dashboard_concat = pd.concat(df_dashoard_hold).reset_index(drop=True)
df_dashboard_concat 

<h2 id="306-get-worksheets-in-dashboard">3.06 Get Worksheets in Dashboard</h2> <a id="306-get-worksheets-in-dashboard"></a> 

##### Get worksheet names associated with a dashboard

In [ ]:
# Function to extract the worksheet names associated with a dashboard
def get_dashboard_worksheets(dashboard):
    # Find all 'zone' elements with the 'name' attribute (assumed to reference worksheets)
    worksheet_references = dashboard.findall(".//zone[@name]")
    worksheet_names = [zone.get('name') for zone in worksheet_references if zone.get('name')]
    
    # Convert to a set to remove duplicates, then back to a list
    unique_worksheet_names = list(set(worksheet_names))
    return unique_worksheet_names

# Find all dashboard elements
dashboards = root.findall(".//dashboard")

df_dashboard_hold = []

for dashboard in dashboards:
    # Extract the dashboard name
    dashboard_name = dashboard.get('name')
    #print(f"Dashboard: {dashboard_name}")
    
    # Extract the list of unique worksheets used in the dashboard
    worksheets_in_dashboard = get_dashboard_worksheets(dashboard)
    
    # Find all 'datasource' elements within the 'datasources' tag under each dashboard
    datasources = dashboard.findall(".//datasources/datasource")
    
    for datasource in datasources:
        # Get the 'caption' or 'name' attribute of the datasource
        caption = datasource.get('caption', 'No Caption')  # Use a default if 'caption' doesn't exist
        name = datasource.get('name', 'No Name')  # Use a default if 'name' doesn't exist
        #print(f"  Datasource: Caption={caption}, Name={name}")
        
        # Find columns in the datasource-dependencies for the current datasource
        datasource_dependencies = dashboard.findall(f".//datasource-dependencies[@datasource='{name}']/column")

        # Handle columns
        for column in datasource_dependencies:
            # Get column attributes
            column_name = re.sub(r'\]|\[', '', column.get('name', 'No Name'))
            column_caption = column.get('caption', 'No Caption')
            column_datatype = column.get('datatype', 'No Datatype')
            column_aggregation = column.get('aggregation', 'No Aggregation')
            column_role = column.get('role', 'No Role')
            column_formula = column.find(".//calculation").get('formula', 'No Formula') if column.find(".//calculation") is not None else 'No Formula'
            
            # Print the column details
            #print(f"    Column Name: {column_name}, Caption: {column_caption}, Datatype: {column_datatype}, Aggregation: {column_aggregation}, Role: {column_role}")
            #print(f"    Formula: {column_formula}")
            
            # Prepare data for dataframe
            data = {
                'type': 'dashboard',
                'title': dashboard_name,
                'datasource': caption,
                'column_name': column_caption,
                'formula': column_formula,
                'worksheets_used': worksheets_in_dashboard  # Keep worksheets as a list
            }
            df_temp1 = pd.DataFrame([data])
            df_dashboard_hold.append(df_temp1)
        
        # Find column instances for the current datasource
        column_instances = dashboard.findall(f".//datasource-dependencies[@datasource='{name}']/column-instance")

        # Handle column instances
        for column_instance in column_instances:
            # Get column-instance attributes
            column_instance_name = column_instance.get('name', 'No Name')
            column_instance_base_column = re.sub(r'\]|\[', '', column_instance.get('column', 'No Base Column'))
            column_instance_derivation = column_instance.get('derivation', 'No Derivation')
            column_instance_type = column_instance.get('type', 'No Type')

            # Print the column-instance details
            #print(f"    Column Instance: {column_instance_name}, Base Column: {column_instance_base_column}, Derivation: {column_instance_derivation}, Type: {column_instance_type}")
            
            # Prepare data for dataframe
            data = {
                'type': 'dashboard',
                'title': dashboard_name,
                'datasource': caption,
                'column_name': column_instance_base_column,
                'worksheets_used': worksheets_in_dashboard  # Keep worksheets as a list
            }
            df_temp2 = pd.DataFrame([data])
            df_dashboard_hold.append(df_temp2)

# Concatenate all dataframes into one
dashboards_with_worksheets_df = pd.concat(df_dashboard_hold).reset_index(drop=True)
dashboards_with_worksheets_df

<h2 id="307-get-xml">3.07 Get XML</h2> <a id="307-get-xml"></a>

##### Get the xml data for each dashboard, worksheet and action

In [ ]:

df_hold = []

# Function to convert an element and its children to a string representation of the XML
def get_xml_string(element):
    return etree.tostring(element, pretty_print=True, encoding='unicode')

# Process all worksheets
worksheets = root.findall(".//worksheet")
for worksheet in worksheets:
    # Extract the title and XML for each worksheet
    worksheet_name = worksheet.get('name')
    worksheet_xml = get_xml_string(worksheet)
    
    # Append to dataframe list
    data = {
        'title': worksheet_name,
        'xml': worksheet_xml,
        'type': 'worksheet'
    }
    df_hold.append(pd.DataFrame([data]))

# Process all dashboards
dashboards = root.findall(".//dashboard")
for dashboard in dashboards:
    # Extract the title and XML for each dashboard
    dashboard_name = dashboard.get('name')
    dashboard_xml = get_xml_string(dashboard)
    
    # Append to dataframe list
    data = {
        'title': dashboard_name,
        'xml': dashboard_xml,
        'type': 'dashboard'
    }
    df_hold.append(pd.DataFrame([data]))



# Find all action elements in the XML
actions = root.findall(".//actions/action")
for action in actions:
    # Extract the source dashboard value
    source_element = action.find(".//source")
    source_dashboard = source_element.get('dashboard', 'Unknown Dashboard') if source_element is not None else 'Unknown Dashboard'
    action_xml = get_xml_string(action)

    # Append to dataframe list
    data = {
        'title': source_dashboard,
        'xml': action_xml,
        'type': 'action'
    }
    df_hold.append(pd.DataFrame([data]))


# Concatenate all the dataframes into a final result
xml_with_products = pd.concat(df_hold).reset_index(drop=True)

# Display the final dataframe
xml_with_products


<h2 id="308-helper-functions">3.08 Helper Functions</h2> <a id="308-helper-functions"></a>

##### For processing support so you can see how things happen

In [150]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def xml_element_parsing(xml_data, max_token=10000, element_type=None, title=None):
    """
    Parses an XML element tree, traversing through its structure, and extracts sections of XML that contain fewer than the specified token threshold.

    The function recursively traverses the provided XML string, evaluates the token count for each element, and collects XML sections that meet the token count criteria. 
    If an element exceeds the token threshold, it continues to parse its child elements until it reaches the smallest section with fewer than the specified token threshold. 
    The function builds a DataFrame containing the extracted sections.

    Parameters:
    xml_data (str): A string containing the XML data to be parsed.
    max_token (int, optional): The maximum number of tokens allowed for each XML section. Defaults to 10,000.
    element_type (str, optional): The type of the XML element. Defaults to None.
    title (str, optional): The title to be used if the element type is 'action'. Defaults to None.

    Parameters:
    max_token (int, optional): The maximum number of tokens allowed for each XML section. Defaults to 10,000.

    Returns:
    pd.DataFrame: A DataFrame with the parsed XML sections, containing the following columns:
        - 'title': The name or title of the element being parsed.
        - 'xml': The XML content of the parsed section.
        - 'xml_element': The XML path to the element (e.g., 'worksheet/table/view').
        - 'type': The type of the XML element being parsed (e.g., 'worksheet').

    Raises:
    ValueError: If the input is not a string or if no sections were found to parse.
    """

    df_hold = []

    # If the input is a string, parse it as XML
    if isinstance(xml_data, str):
        root = etree.fromstring(xml_data)
    else:
        raise ValueError("Input must be a string containing XML data.")

    # Function to convert an element and its children to a string representation of the XML
    def get_xml_string(element):
        return etree.tostring(element, pretty_print=True, encoding='unicode')

    # Recursive function to parse XML sections based on token count threshold
    def parse_element(element, xml_path, title):
        element_xml = get_xml_string(element)
        token_count = num_tokens_from_string(element_xml, "cl100k_base")

        if token_count < max_token:
            data = {
                'title': title,
                'xml': element_xml,
                'xml_element': xml_path,
                'type': element.tag,
                'token_count':token_count
            }
            df_hold.append(data)
        else:
            # If token count exceeds threshold, go deeper into child elements
            for child in element:
                child_path = f"{xml_path}/{child.tag}"
                parse_element(child, child_path, title)

    # Start parsing from the root element
    element_name = title if element_type == 'action' and title else root.get('name', root.tag)
    parse_element(root, root.tag, element_name)

    # Create a dataframe from the collected data
    if df_hold:
        xml_with_products = pd.DataFrame(df_hold).reset_index(drop=True)
    else:
        raise ValueError("No sections found to parse.")

    return xml_with_products



def genAI_summarize_worksheet(text, api_key, model):
    # API endpoint URL
    api_url = 'https://api.openai.com/v1/chat/completions'

    # Request headers
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }


    data = {
        'model': model,
        'messages': [{'role': 'system', 'content': '''
    You are a data analyst writing documentation to summarize a Tableau worksheet. 
    Your writing style should be able to be understood by senior leadership executives along with operational teams'''},
                        {'role': 'user', 'content': f'''
    You are going to be provided parsed XML data from a Tableau worksheet or summarize parts of the XML in that worksheet which contain information critical to translating this metadata into 
    information about the worksheets which include the data sources and fields used. Your summariztion should be user friendly descriptive that provides details to 
    end users. 

    To ensure the required details are provided you must include the following:
    1) The title of worksheet.
    2) Datasource and fields, if available 
    3) Any available filters or other controls.
    3) Any calculated fields and how they are derived.
    4) Other details you may find interesting or helpful. 
                        
    Start Parsed XML Data:
    {text}           
    '''}]
    }

    # Make the API call
    response = requests.post(api_url, json=data, headers=headers)

    # Process the API response
    if response.status_code == 200:
        print('success')
        result = response.json()
        chatgpt_response = result['choices'][0]['message']['content']

    else:
        print('error')
        chatgpt_response = f"'Error:', {response.status_code}, {response.text}"

    return chatgpt_response





def genAI_summarize_worksheet_elements(text, api_key, model):
    # API endpoint URL
    api_url = 'https://api.openai.com/v1/chat/completions'

    # Request headers
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }


    data = {
        'model': model,
        'messages': [{'role': 'system', 'content': '''
    You are a data analyst writing documentation to summarize what particular XML elements do and why they are important in a Tableau worksheet. 
    Your writing style should be able to be understood by senior leadership executives along with operational teams'''},
                        {'role': 'user', 'content': f'''
    You are going to be provided parsed XML data from particular XML elements in a Tableau worksheet which contain information critical to translating this metadata into 
    information about the worksheets which include the data sources and fields used. 
    Your goal is to provide a summariztion that will be added to other summarized sections that will be then be used to summarize the entire workbook.  

    To ensure the required details are provided you must include the following:
    1) The title of worksheet.
    2) Datasource and fields, if available 
    3) Any available filters or other controls.
    3) Any calculated fields and how they are derived.
    4) Other details you may find interesting or helpful. 
                        
    Start Parsed XML Data:
    {text}           
    '''}]
    }

    # Make the API call
    response = requests.post(api_url, json=data, headers=headers)

    # Process the API response
    if response.status_code == 200:
        print('success')
        result = response.json()
        chatgpt_response = result['choices'][0]['message']['content']

    else:
        print('error')
        chatgpt_response = f"'Error:', {response.status_code}, {response.text}"

    return chatgpt_response



def genAI_summarize_view(text, api_key, model):
    # API endpoint URL
    api_url = 'https://api.openai.com/v1/chat/completions'

    # Request headers
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }


    data = {
        'model': model,
        'messages': [{'role': 'system', 'content': '''
    You are a data analyst writing documentation to summarize a Tableau view which could be a dashboard or a worksheet. 
    Your writing style should be able to be understood by senior leadership executives along with operational teams'''},
                        {'role': 'user', 'content': f'''
    You are going to be provided parsed XML data from particular Tableau view which could be a dashboard or a worksheet.
    If the view is a dashboard you will also be provided with summarized content for the worksheets that support this dashboard which need to be incorported into your response.   
    Your goal is to provide a a complete summarization of the view that meets the following conditions.  

    To ensure the required details are provided you must include the following:
    1) The title of worksheet.
    2) Datasource and fields, if available 
    3) Any available filters or other controls.
    3) Any calculated fields and how they are derived.
    4) Other details you may find interesting or helpful. 
                        
    Start Parsed XML Data:
    {text}           
    '''}]
    }

    # Make the API call
    response = requests.post(api_url, json=data, headers=headers)

    # Process the API response
    if response.status_code == 200:
        print('success')
        result = response.json()
        chatgpt_response = result['choices'][0]['message']['content']

    else:
        print('error')
        chatgpt_response = f"'Error:', {response.status_code}, {response.text}"

    return chatgpt_response




<h2 id="309-preprocessing">3.09 Preprocessing</h2> <a id="309-preprocessing"></a>

##### Preprocess the dashboards, worksheets and actions to ensure the target amount of token in the XML are ready for AI processing

In [ ]:
# get the xml meta from the published views, only dashboard and worksheets, and determine the tokens in that view
views_xml_records = xml_with_products[(xml_with_products.title.isin(find_views)) & (xml_with_products.type != 'action')]


# get the actions that are scoped to the views 
view_actions = xml_with_products[(xml_with_products.title.isin(find_views)) & (xml_with_products.type == 'action')].drop_duplicates().reset_index(drop=True)


# get a list of the views that are dashboards and use them as a filter to get a unique listing of worksheets
dashboard_views_xml_records = views_xml_records[views_xml_records.type == 'dashboard']
dashboard_list = list(set(list(dashboard_views_xml_records.title)))


# get a unique listing based on the dashboard title so a unique listing of the worksheets_used can be derived
worksheets_in_views = dashboards_with_worksheets_df[dashboards_with_worksheets_df.title.isin(dashboard_list)].drop_duplicates(subset=('title'), keep='first')

worksheet_big_list = list(worksheets_in_views.worksheets_used)

# loop through each list pull the items and make one large list
worksheet_hold_list = []
for sheet_list in worksheet_big_list:
    for sheet in sheet_list:
        #print(sheet)
        worksheet_hold_list.append(sheet)
        
final_worksheet_list = list(set(worksheet_hold_list))

# get a listing of the worksheets that are used in the dashboards and determine their token count
worksheet_xml_records = xml_with_products[xml_with_products.title.isin(final_worksheet_list)]



# loop through views, actions and worksheets and parse the required XML
views_xml_records_hold = []
for view_row in views_xml_records.itertuples():
    views_xml_records_hold.append(xml_element_parsing(view_row.xml))

views_xml_records_token_processed = pd.concat(views_xml_records_hold).drop_duplicates(subset={'title','xml','type'}, keep='first').reset_index(drop=True)


view_actions_hold = []
for actions_row in view_actions.itertuples():
    view_actions_hold.append(xml_element_parsing(actions_row.xml, max_token=10000, element_type='action', title=actions_row.title))

view_actions_token_processed = pd.concat(view_actions_hold).drop_duplicates(subset={'title','xml','type'}, keep='first').reset_index(drop=True)


worksheet_xml_records_hold = []
for worksheet_row in worksheet_xml_records.itertuples():
    worksheet_xml_records_hold.append(xml_element_parsing(worksheet_row.xml))

worksheet_xml_records_token_processed = pd.concat(worksheet_xml_records_hold).drop_duplicates(subset={'title','xml','type'}, keep='first').reset_index(drop=True)

<h2 id="310-ai-preprocessing">3.10 AI Processing</h2> <a id="310-ai-preprocessing"></a>

1. ##### The code is setup for testing to save money on AI API calls so read the comments in the code to update if you want to run without restrictions. 
2. ##### The first code block processes worksheets while the next processes dashboards to derive the final product 

In [ ]:
######
### comment out scoped_test_worksheets_list and replace with the variable unique_worksheet_list below when not testing to save $$$ on API calls 
#####
# select one dashboard to process with all required worksheets
scoped_test_worksheets_list = dashboards_with_worksheets_df[dashboards_with_worksheets_df.title == views_xml_records_token_processed.iloc[3].title].iloc[0].worksheets_used
scoped_test_worksheets_list = [item.strip() for item in scoped_test_worksheets_list]


# determine if there are any worksheets as views and add them to the list of worksheets for AI processing 
worksheet_views_xml_records = views_xml_records_token_processed[views_xml_records_token_processed.type == 'worksheet']

if len(worksheet_views_xml_records):
    worksheet_xml_records_token_processed = pd.concat([worksheet_xml_records_token_processed,worksheet_views_xml_records]).reset_index(drop=True)


# remove the xml_element value 'worksheet/simple-id' from the worksheets data as this add no value for users
worksheet_xml_records_token_processed =  worksheet_xml_records_token_processed[worksheet_xml_records_token_processed.xml_element != 'worksheet/simple-id'].reset_index(drop=True)


# get a unique list of worksheets to iterage for AI processing
unique_worksheet_list = list(set(list(worksheet_xml_records_token_processed.title.str.strip())))







ai_df_hold = []

# iterate over each worksheet and if the total amount of tokens is more than the target then process in chunks
for sheet in scoped_test_worksheets_list:#unique_worksheet_list:  <---------------------------------------------------- change the variable to unique_worksheet_list when fully running

    # get the target worksheet
    worksheet_target = worksheet_xml_records_token_processed[worksheet_xml_records_token_processed.title.str.strip() == sheet]
    
    # chunk out processing if above target token count
    if worksheet_target.token_count.sum() >= 10000:

        # Grouping by xml_element and summing token_count
        grouped_df = worksheet_target.groupby('xml_element', as_index=False)['token_count'].sum()

        # Adding a new field 'processing_group' to group records by cumulative token count
        # Goal of this is to group xml_element under the target token around while also 
        # identifing xml_element that are above so they can be parsed seperately 
        process_group = 0
        running_total = 0
        process_groups = []

        for _, row in grouped_df.iterrows():
            if running_total + row['token_count'] >= 10000:
                process_group += 1
                running_total = 0
            running_total += row['token_count']
            process_groups.append(process_group)

        grouped_df['processing_group'] = process_groups

        # get a list of the processing groups to pair accordingly 
        processing_list = list(set(list(grouped_df.processing_group)))

        xml_ai_content = f'''
        Worksheet Title: {sheet}
        Worksheet Summarization:
        '''

        for g in processing_list:
            process_rows = grouped_df[grouped_df.processing_group == g]

            # if there is only 1 row then this indictes the xml_element	group 
            # is over the target token amount and needs to have the subsections summarized
            if len(process_rows) == 1 and process_rows.iloc[0].token_count >= 10000:
                #
                print(f'{process_rows.iloc[0].xml_element} <-- element above 10k for processing')
                print(f'{worksheet_target.iloc[0].title} <-- worksheet name')
                print('')
                element_target_rows = worksheet_target[worksheet_target.xml_element == process_rows.iloc[0].xml_element] 
                
                # Adding a new field 'processing_group' to group records by cumulative token count
                process_group = 0
                running_total = 0
                process_groups = []

                for _, row in element_target_rows.iterrows():
                    if running_total + row['token_count'] >= 10000:
                        process_group += 1
                        running_total = 0
                    running_total += row['token_count']
                    process_groups.append(process_group)

                element_target_rows['processing_group'] = process_groups           
                
                element_processing_list = list(set(list(element_target_rows.processing_group)))

                # get the processing group rows as a set
                for e in element_processing_list:
                    e_target_rows = element_target_rows[element_target_rows.processing_group == e] 

                    # create a processing group string then send to AI to summerize
                    xml_group_ai_content = f'''
                    Worksheet Title: {worksheet_target.iloc[0].title}
                    XML Element Content:
                    '''

                    # iterate over the process group rows and add the xml to the ai content
                    for e_row in e_target_rows.itertuples():
                        xml_group_ai_content = xml_group_ai_content + e_row.xml

                    xml_ai_content = xml_ai_content + genAI_summarize_worksheet_elements(xml_group_ai_content, api_key, model)

            else:
                # use the xml_element field in the grouped_df to create a list to filter the 
                # the required rows to use the XML from those row to build the AI prompt input
                xml_element_list = list(set(list(process_rows.xml_element)))

                xml_rows = worksheet_target[worksheet_target.xml_element.isin(xml_element_list)]

                # create a processing group string then send to AI to summerize
                xml_group_ai_content = f'''
                Worksheet Title: {worksheet_target.iloc[0].title}
                XML Element Content:
                '''

                for x_row in xml_rows.itertuples():
                    xml_group_ai_content = xml_group_ai_content + x_row.xml

                xml_ai_content = xml_ai_content + genAI_summarize_worksheet_elements(xml_group_ai_content, api_key, model)

        # get ai content and add to a dataframe for downstream processing
        final_sum_worksheet = genAI_summarize_worksheet(xml_ai_content, api_key, model)
        ai_data = {'title':sheet,
                   'ai_summary':final_sum_worksheet }
        
        ai_temp = pd.DataFrame([ai_data])
        ai_df_hold.append(ai_temp)

    else:
        
        # create a processing group string then send to AI to summerize
        xml_group_ai_content = f'''
        Worksheet Title: {sheet}
        XML Element Content:
        '''
        for w_row in worksheet_target.itertuples():
            xml_group_ai_content = xml_group_ai_content + w_row.xml


        # get ai content and add to a dataframe for downstream processing
        final_sum_worksheet = genAI_summarize_worksheet(xml_group_ai_content, api_key, model)
        ai_data = {'title':sheet,
                   'ai_summary':final_sum_worksheet }

        ai_temp = pd.DataFrame([ai_data])
        ai_df_hold.append(ai_temp)


worksheet_ai_content = pd.concat(ai_df_hold).reset_index(drop=True)
 
print(worksheet_ai_content.iloc[0].ai_summary)

> **Action**: Pay attention to the code comments for instructions regarding testing

In [ ]:
# select one dashboard to process with all required worksheets that have already been ai processed 
# After testing, uncomment view_to_process and scoped_test_worksheets_list and comment out the following two lines as these are only using one dashboard for testing
view_to_process = views_xml_records_token_processed[views_xml_records_token_processed.title == views_xml_records_token_processed.iloc[3].title]
scoped_test_worksheets_list = dashboards_with_worksheets_df[dashboards_with_worksheets_df.title == views_xml_records_token_processed.iloc[3].title].iloc[0].worksheets_used

#### -------->>>>  UNCOMMENT the text below after testing   <<<<-------- 
#view_to_process = views_xml_records_token_processed.copy()
#scoped_test_worksheets_list = dashboards_with_worksheets_df.copy()


scoped_test_worksheets_list = [item.strip() for item in scoped_test_worksheets_list]




view_ai_prep_hold = []

for view in view_to_process.itertuples():

    
    
    # if the record is a dashboard, then include the worksheets in the build process
    view_type = view.type

    ai_content = f'''
Tableau Workbook Name: {source_workbook['Workbook Name']}, 
Tableau Workbook URL:  {source_workbook['Workbook Webpage_Url']}, 
Workbook View Name:    {view.title},
Workbook View Type:    {view.type},
{view.type} XML:       {view.xml}
Included Worksheet(s) AI Summarization:        
'''


    if view_type == 'dashboard':
        # get the worksheets in the view
        worksheets_in_views = dashboards_with_worksheets_df[dashboards_with_worksheets_df.title == view.title].iloc[0].worksheets_used
        worksheets_in_views_list = [item.strip() for item in worksheets_in_views]

        
        ai_worksheet_rows = worksheet_ai_content[worksheet_ai_content.title.isin(worksheets_in_views_list)]
        # iterate over the worksheets and add the data to the ai_content string
        for row in ai_worksheet_rows.itertuples():
            # create worksheet content and add to the ai string
            worksheet_content = f'''
Worksheet Name: {row.title},
Worksheet AI Summarization:  {row.ai_summary}
'''
            ai_content = ai_content+worksheet_content

    # get ai content and add to a dataframe for downstream processing
    final_sum_view = genAI_summarize_view(ai_content, api_key, model)
    ai_data = {'title':view.title,
                'ai_summary':final_sum_view}
    
    ai_temp = pd.DataFrame([ai_data])
    view_ai_prep_hold.append(ai_temp)    


      
view_ai_preprocessed = pd.concat(view_ai_prep_hold).reset_index(drop=True)
print(view_ai_preprocessed.iloc[0].ai_summary)


